In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as wavfile
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from joblib import Parallel, delayed
from tqdm import tqdm



In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load datasets
dataset_path = r"C:\Users\azatv\Jupyter\JupyterProjects\ML and PC\development.csv"
feature_names_path = r"C:\Users\azatv\Jupyter\JupyterProjects\ML and PC\idx_to_feature_name.csv"
timestamp_dataset_path = r"C:\Users\azatv\Jupyter\JupyterProjects\ML and PC\development.npy"

development = pd.read_csv(dataset_path)
idx_to_feature_name = pd.read_csv(feature_names_path)
timestamp_dataset = np.load(timestamp_dataset_path)

labels = development['word']

reduced_size = len(development) // 100

unique_labels = labels.unique()
samples_per_label = reduced_size // len(unique_labels)

reduced_indices = []

for label in unique_labels:
    label_indices = development[development['word'] == label].index.tolist()
    sampled_indices = np.random.choice(label_indices, samples_per_label)
    reduced_indices.extend(sampled_indices)

reduced_development = development.loc[reduced_indices].reset_index(drop=True)
reduced_timestamp_dataset = timestamp_dataset[reduced_indices]

print(f"Reduced development shape: {reduced_development.shape}")
print(f"Reduced timestamp dataset shape: {reduced_timestamp_dataset.shape}")

Reduced development shape: (441, 4)
Reduced timestamp dataset shape: (441, 175, 44)


In [3]:
labels_repeated = reduced_development['word'].repeat(44).reset_index(drop=True)

reduced_timestamp_dataset = np.swapaxes(reduced_timestamp_dataset, 1, 2)
x, y, z = reduced_timestamp_dataset.shape  
print(f"x,y,z :{x,y,z}")

reduced_timestamp_dataset_xyz = reduced_timestamp_dataset.reshape((x * y, z))

print(reduced_timestamp_dataset_xyz.shape)
df = pd.DataFrame(reduced_timestamp_dataset_xyz)
df.columns = list(idx_to_feature_name['feature_name'])
df['label'] = labels_repeated

print(df.shape)
print(df.head())

x,y,z :(441, 44, 175)
(19404, 175)
(19404, 176)
   bandwidth_0   centroid_0  contrast_0  contrast_1  contrast_2  contrast_3  \
0  3293.925414  1904.772044    3.254015    2.145693    4.591402    8.548419   
1  3401.407879  2003.966470    5.548540    7.209587    6.677888   10.362639   
2  3101.558996  1374.951201    6.986853    2.873223   11.016584   11.755014   
3  3130.988887  1459.063982    8.080800    3.305663    3.508137    7.832932   
4  3252.981235  1716.807020    6.909907    8.106370    7.700668    7.229105   

   contrast_4  contrast_5  contrast_6  energy_0  ...  mfcc_d2_26  mfcc_d2_27  \
0   14.942765   10.686919   12.397009  2.588598  ...   -0.007337   -0.055169   
1    9.464603   12.222898   11.917046  2.778460  ...   -0.007337   -0.055169   
2    8.432297   14.832628   13.527659  3.797681  ...   -0.007337   -0.055169   
3    9.460101   18.393203   18.549774  3.190613  ...   -0.007337   -0.055169   
4   10.156537    9.865179   11.376497  2.861669  ...   -0.007337   -0.055169 

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

X = df.drop(columns=['label'])
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

Training set shape: (15523, 175)
Testing set shape: (3881, 175)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as wavfile
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from joblib import Parallel, delayed
from tqdm import tqdm

In [6]:
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
reg.fit(X, y)
importance = reg.feature_importances_

In [8]:
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': importance})

feature_importances = feature_importances.sort_values(by='importance', ascending=False)

print(feature_importances)

threshold = 0.01
important_features = feature_importances[feature_importances['importance'] > threshold]['feature'].tolist()

print(f'Selected important features: {important_features}')

         feature  importance
82        mfcc_6    0.025790
109     mfcc_d_1    0.016628
108     mfcc_d_0    0.014872
110     mfcc_d_2    0.014722
143    mfcc_d2_3    0.014689
..           ...         ...
69   melspect_57    0.002094
37   melspect_25    0.002032
48   melspect_36    0.001981
46   melspect_34    0.001972
76        mfcc_0    0.001546

[175 rows x 2 columns]
Selected important features: ['mfcc_6', 'mfcc_d_1', 'mfcc_d_0', 'mfcc_d_2', 'mfcc_d2_3', 'mfcc_d_4', 'mfcc_d_7', 'mfcc_d_3', 'mfcc_d2_1', 'mfcc_d_6', 'mfcc_d_27', 'mfcc_d2_0', 'mfcc_d_5', 'mfcc_d2_6']


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

selected_features = df[important_features]
labels = df['label']

X_train, X_temp, y_train, y_temp = train_test_split(selected_features, labels, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

scaler = StandardScaler()
X_train_selected = scaler.fit_transform(X_train)
X_val_selected = scaler.transform(X_val)
X_test_selected = scaler.transform(X_test)

## Classifiers

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

classifiers = {
    'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
    'SVM': SVC(random_state=42),
    'KNN': KNeighborsClassifier()
}

param_grids = {
    'RandomForest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
    },
    'SVM': {
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto'],
        'kernel': ['linear', 'rbf']
    },
    'KNN': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
    }
}

best_classifiers = {}
for clf_name, clf in classifiers.items():
    print(f"Training {clf_name}...")
    print(f"Clf: {clf}")
    grid_search = GridSearchCV(clf, param_grids[clf_name], cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train_selected, y_train)
    best_classifiers[clf_name] = grid_search.best_estimator_
    print(f"Best {clf_name} parameters: {grid_search.best_params_}")
    print(f"Best {clf_name} cross-validation accuracy: {grid_search.best_score_}")

for clf_name, clf in best_classifiers.items():
    val_accuracy = clf.score(X_val_selected, y_val)
    print(f"{clf_name} validation accuracy: {val_accuracy}")

Training RandomForest...
Clf: RandomForestClassifier(n_jobs=-1, random_state=42)
Best RandomForest parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
Best RandomForest cross-validation accuracy: 0.5799704237176849
Training SVM...
Clf: SVC(random_state=42)
Best SVM parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Best SVM cross-validation accuracy: 0.45430276256789115
Training KNN...
Clf: KNeighborsClassifier()
Best KNN parameters: {'algorithm': 'auto', 'n_neighbors': 3, 'weights': 'distance'}
Best KNN cross-validation accuracy: 0.5755894968417047
RandomForest validation accuracy: 0.607317701623293
SVM validation accuracy: 0.45581035815511467
KNN validation accuracy: 0.6189126513785107


In [13]:
from sklearn.metrics import accuracy_score, classification_report
for clf_name, clf in best_classifiers.items():
    test_accuracy = clf.score(X_test_selected, y_test)
    y_pred = clf.predict(X_test_selected)
    report = classification_report(y_test, y_pred)
    print(f"{clf_name} test accuracy: {test_accuracy}")
    print(f"{clf_name} classification report:\n{report}")

RandomForest test accuracy: 0.6137593403761917
RandomForest classification report:
              precision    recall  f1-score   support

           0       0.58      0.58      0.58       190
           1       0.59      0.68      0.63       149
           2       0.70      0.77      0.74       201
           3       0.57      0.64      0.60       176
           4       0.69      0.51      0.58       195
           5       0.61      0.63      0.62       180
           6       0.58      0.68      0.63       177
           7       0.70      0.60      0.65       184
           8       0.73      0.53      0.62       192
           9       0.63      0.68      0.66       185
          10       0.66      0.60      0.63       188
          11       0.54      0.60      0.57       177
          12       0.61      0.62      0.62       188
          13       0.55      0.57      0.56       187
          14       0.58      0.51      0.54       171
          15       0.52      0.55      0.54       18

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

dataset_path = r'C:\Users\azatv\Jupyter\JupyterProjects\ML and PC\new_df30.1.csv'
df = pd.read_csv(dataset_path, index_col=0)

X = df.drop(columns=['label']).values
y = df['label'].values

# Check if GPU is available and set it up
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        print(physical_devices)
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        tf.config.experimental.set_virtual_device_configuration(
            physical_devices[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]
        )
        print(f"Using GPU: {physical_devices[0].name}")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found. Using CPU.")

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_onehot = to_categorical(y_encoded)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y_onehot, test_size=0.2, stratify=y_onehot, random_state=42)

model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1]))
model.add(Activation('sigmoid'))
model.add(Dense(64))
model.add(Activation('sigmoid'))
model.add(Dense(y_onehot.shape[1]))
model.add(Activation('softmax'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_valid, y_valid), verbose=1)

train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
print(f"Training Accuracy: {train_accuracy*100:.2f}%")

valid_loss, valid_accuracy = model.evaluate(X_valid, y_valid, verbose=0)
print(f"Validation Accuracy: {valid_accuracy*100:.2f}%")

model_path = 'linear-softmax0.2.keras'
model.save(model_path)
print("Model saved successfully.")

pred_probs = model.predict(X_valid)
pred_labels = np.argmax(pred_probs, axis=1)
true_labels = np.argmax(y_valid, axis=1)

valid_accuracy = np.mean(pred_labels == true_labels)
print(f"Validation Accuracy: {valid_accuracy*100:.2f}%")

print(classification_report(true_labels, pred_labels, target_names=label_encoder.classes_))



# output:

# [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
# Using GPU: /physical_device:GPU:0
# Epoch 1/50
# 1641/1641 [==============================] - 5s 2ms/step - loss: 2.8302 - accuracy: 0.1291 - val_loss: 2.7018 - val_accuracy: 0.1670
# Epoch 2/50
# 1641/1641 [==============================] - 3s 2ms/step - loss: 2.6455 - accuracy: 0.1824 - val_loss: 2.6053 - val_accuracy: 0.1959
# Epoch 3/50
# 1641/1641 [==============================] - 4s 2ms/step - loss: 2.5481 - accuracy: 0.2115 - val_loss: 2.5355 - val_accuracy: 0.2139
# Epoch 4/50
# 1641/1641 [==============================] - 4s 2ms/step - loss: 2.4569 - accuracy: 0.2396 - val_loss: 2.4545 - val_accuracy: 0.2356
# Epoch 5/50
# 1641/1641 [==============================] - 4s 2ms/step - loss: 2.3634 - accuracy: 0.2696 - val_loss: 2.3792 - val_accuracy: 0.2595
# Epoch 6/50
# 1641/1641 [==============================] - 4s 2ms/step - loss: 2.2752 - accuracy: 0.2960 - val_loss: 2.3141 - val_accuracy: 0.2834
# Epoch 7/50
# 1641/1641 [==============================] - 4s 2ms/step - loss: 2.1918 - accuracy: 0.3190 - val_loss: 2.2625 - val_accuracy: 0.2927
# Epoch 8/50
# 1641/1641 [==============================] - 4s 2ms/step - loss: 2.1186 - accuracy: 0.3409 - val_loss: 2.1968 - val_accuracy: 0.3133
# Epoch 9/50
# 1641/1641 [==============================] - 4s 2ms/step - loss: 2.0528 - accuracy: 0.3576 - val_loss: 2.1601 - val_accuracy: 0.3185
# Epoch 10/50
# 1641/1641 [==============================] - 4s 2ms/step - loss: 1.9957 - accuracy: 0.3741 - val_loss: 2.1216 - val_accuracy: 0.3311
# Epoch 11/50
# 1641/1641 [==============================] - 4s 2ms/step - loss: 1.9442 - accuracy: 0.3900 - val_loss: 2.0959 - val_accuracy: 0.3363
# Epoch 12/50
# ...
#     accuracy                           0.39     13121
#    macro avg       0.39      0.39      0.39     13121
# weighted avg       0.39      0.39      0.39     13121


## Wav test

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import librosa
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

model_path = 'linear-softmax0.2.keras'
model = load_model(model_path)

feature_names_path = r'C:\Users\azatv\Jupyter\JupyterProjects\ML and PC\idx_to_feature_name.csv'
feature_names_df = pd.read_csv(feature_names_path)
expected_features = feature_names_df['feature_name'].tolist()

def extract_features(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None)
        if len(y) < 512:
            raise ValueError("Audio file too short")

        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)[:, :44]
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)[:, :44]
        mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40)[:, :44]
        contrast = librosa.feature.spectral_contrast(y=y, sr=sr)[:, :44]
        tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr)[:, :44]

        features = np.concatenate((mfccs.flatten(), chroma.flatten(), mel.flatten(), contrast.flatten(), tonnetz.flatten()))

        # Ensure total number of features is 174
        if len(features) > 174:
            features = features[:174]
        else:
            features = np.pad(features, (0, 174 - len(features)), 'constant')

        return features
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

dataset_dir = r'C:\Users\azatv\Jupyter\JupyterProjects\ML and PC\MLPC24_speech_commands_raw_waveforms'

def prepare_test_data(dataset_dir, num_samples=10):
    test_data = []
    test_labels = []
    labels = [label for label in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, label))]
    for label in labels:
        label_dir = os.path.join(dataset_dir, label)
        files = [f for f in os.listdir(label_dir) if not f.startswith("._")]
        random_files = random.sample(files, min(num_samples, len(files)))
        for file in random_files:
            file_path = os.path.join(label_dir, file)
            features = extract_features(file_path)
            if features is not None:
                test_data.append(features)
                test_labels.append(label)
    return np.array(test_data), np.array(test_labels)

X_test, y_test = prepare_test_data(dataset_dir)

print(f"Feature shape: {X_test.shape}")

if X_test.shape[0] == 0:
    raise ValueError("No valid test data found")

scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

label_encoder = LabelEncoder()
y_test_encoded = label_encoder.fit_transform(y_test)
y_test_onehot = to_categorical(y_test_encoded)

pred_probs = model.predict(X_test_scaled)
pred_labels = np.argmax(pred_probs, axis=1)
true_labels = np.argmax(y_test_onehot, axis=1)

accuracy = np.mean(pred_labels == true_labels)
print(f"Test Accuracy: {accuracy*100:.2f}%")

print(classification_report(true_labels, pred_labels, target_names=label_encoder.classes_))


# output:
# Feature shape: (210, 174)
# 7/7 [==============================] - 1s 1ms/step
# Test Accuracy: 3.33%
#               precision    recall  f1-score   support

#        Alarm       0.00      0.00      0.00        10
#     Brötchen       0.00      0.00      0.00        10
#    Fernseher       0.14      0.10      0.12        10
#         Haus       0.00      0.00      0.00        10
#      Heizung       0.00      0.00      0.00        10
#      Leitung       0.08      0.10      0.09        10
#        Licht       0.00      0.00      0.00        10
#      Lüftung       0.18      0.20      0.19        10
#         Ofen       0.00      0.00      0.00        10
#        Radio       0.00      0.00      0.00        10
#     Schraube       0.00      0.00      0.00        10
#      Spiegel       0.00      0.00      0.00        10
#  Staubsauger       0.09      0.10      0.10        10
#           an       0.17      0.10      0.12        10
#          aus       0.00      0.00      0.00        10
#         kann       0.00      0.00      0.00        10
#        nicht       0.00      0.00      0.00        10
#        offen       0.00      0.00      0.00        10
#        other       0.07      0.10      0.08        10
#         warm       0.00      0.00      0.00        10
#    wunderbar       0.00      0.00      0.00        10

#     accuracy                           0.03       210
#    macro avg       0.04      0.03      0.03       210
# weighted avg       0.04      0.03      0.03       210